In [ ]:
# +============================================================================+
# | Company:   SOFiSTiK AG                                                     |
# | Version:   SOFiSTIK 2022                                                   |
# +============================================================================+
from sofistik_daten import *
import os               # for the environment variable necessary, this is a great tool
import platform         # checks the python platform
import string
from ctypes import *    # read the functions from the cdb
import math
import pandas as pd

In [ ]:
# This example has been tested with Python 3.7 (64-bit)
##print ("The path variable=", os.environ["Path"])

# Check the python platform (32bit or 64bit)
##print ("Python architecture=", platform.architecture())
sofPlatform = str(platform.architecture())



# Set DLL dir path - new in PY 3.8 for ctypes
# See: https://docs.python.org/3/whatsnew/3.8.html#ctypes
os.add_dll_directory(r"C:\Program Files\SOFiSTiK\2022\SOFiSTiK 2022\interfaces\64bit")
os.add_dll_directory(r"C:\Program Files\SOFiSTiK\2022\SOFiSTiK 2022")

# Get the DLL functions
myDLL = cdll.LoadLibrary(r"C:\Program Files\SOFiSTiK\2022\SOFiSTiK 2022\sof_cdb_w-2022.dll")
py_sof_cdb_get = cdll.LoadLibrary("sof_cdb_w-2022.dll").sof_cdb_get
py_sof_cdb_get.restype = c_int

py_sof_cdb_kenq = cdll.LoadLibrary("sof_cdb_w-2022.dll").sof_cdb_kenq_ex


# Connect to CDB
Index = c_int()
cdbIndex = 99

# input the cdb path here
fileName = r"C:\Users\nooyur\Documents\01 Arbeidsmappe\01 Prosjektmappe\03 Betna - Hestnes\Beregninger\Sofistik\Hestnessletta_V5\Hestnessletta.cdb"

# important: Unicode call!
Index.value = myDLL.sof_cdb_init(fileName.encode('utf8'), cdbIndex)

# get the CDB status
cdbStat = c_int()
cdbStat.value = myDLL.sof_cdb_status(Index.value)

# Print the Status of the CDB
print ("CDB Status:", cdbStat.value)

pos = c_int(0)
datalen = c_int(0)

a = c_int
ie = c_int()

datalen.value = sizeof(CNODE)
RecLen = c_int(sizeof(cnode))

"""
do while ie == 0, see cdbase.chm, Returnvalue.
   = 0 -> No error
   = 1 -> Item is longer than Data
   = 2 -> End of file reached
   = 3 -> Key does not exist
"""
"""
while ie.value < 2:
   ie.value = py_sof_cdb_get(Index, 20, 0, byref(cnode), byref(RecLen), 1)
   print("{:10d}{:10d}{:10d}{:10d}{:10.2f}{:10.2f}{:10.2f}".format(
      cnode.m_nr,      # node-number
      cnode.m_inr,     # internal node-number
      cnode.m_kfix,    # degree of freedoms
      cnode.m_ncod,    # additional bit code
      cnode.m_xyz[0],  # x coordinates
      cnode.m_xyz[1],  # y coordinates
      cnode.m_xyz[2])  # z coordinates
   ) 

loadcase = 1

forces = []
RecLen = c_int(sizeof(cbeam_foc))
while ie.value == 0:
   ie.value = py_sof_cdb_get(Index, 102, loadcase, byref(cbeam_foc), byref(RecLen), 1) 
   #if cbeam_foc.m_id == 200007:
   #   print(cbeam_foc.m_n)
   if 200001 <= cbeam_foc.m_id <= 200007: 
      forces.append(math.ceil(cbeam_foc.m_n))
      #print(math.ceil(cbeam_foc.m_n))
   
RecLen = c_int(sizeof(cbeam_foc))
myDLL.sof_cdb_flush(Index.value)
#print(forces)
"""


datalen.value = sizeof(CNODE)
RecLen = c_int(sizeof(cnode))

#print(datalen.value)


In [ ]:
"""
CGPT(Structure):             # 30/NR:0  Structural points
   _fields_ = [
         ('m_id', c_int),          #        identifier 0 (always the first record)
         ('m_ibc', c_int),         #        Boundary condition
         ('m_xyz', c_float * 3),   # [1001] Ordinates
         ('m_t', c_float * 3 * 3), #        transformation matrix
         ('m_bx', c_float),        # [1001] Dimension in x
         ('m_by', c_float),        # [1001] Dimension in y
         ('m_thick', c_float),     # [1010] Plate thickness
         ('m_hmesh', c_float),     # [1001] Mesh density at point
         ('m_r17', c_float),       #        reserved
         ('m_text',  c_int * 17)   #        Designation of Point
"""       
       
def GetStructuralPoints(cdb_link):
    # Connect to CDB
    Index = c_int()
    cdbIndex = 99

    # input the cdb path here
    fileName = cdb_link

    # important: Unicode call!
    Index.value = myDLL.sof_cdb_init(fileName.encode('utf8'), cdbIndex)

    # get the CDB status
    cdbStat = c_int()
    cdbStat.value = myDLL.sof_cdb_status(Index.value)

    # Print the Status of the CDB
    print ("CDB Status:", cdbStat.value)

    pos = c_int(0)
    datalen = c_int(0)

    a = c_int
    ie = c_int()

    #datalen.value = sizeof(CNODE)
    datalen.value = sizeof(CGPT)
    RecLen = c_int(sizeof(cgpt))
    
    NodeID = []
    
    
    while ie.value < 2:
        ie.value = py_sof_cdb_get(Index, 30, 0, byref(cgpt), byref(RecLen), 1)
        NodeID.append(cgpt.m_id)
            
    
        #Data = {'NodeNum' : NodeNum, 'IntNodeNum' : IntNodeNum,'X_Coor' : X_Coor,'Y_Coor' : Y_Coor,'Z_Coor' : Z_Coor}
        RecLen = c_int(sizeof(cgpt))
        myDLL.sof_cdb_close(0)
        cdbStat.value = myDLL.sof_cdb_status(Index.value)
    
    return NodeID